In [1]:
from pyannote.database import registry
import os
registry.load_database(
    os.path.join('/mnt/', 'e', 'Files', 'Acoustic_Data', 'Datasets_specified', 'yaml','My_Databases_specified.yml')
    )
os.environ["PYANNOTE_DATABASE_CONFIG"] = os.path.join('/mnt/', 'e', 'Files', 'Acoustic_Data', 'Datasets_specified','yaml', 'My_Databases_specified.yml')

from pyannote.database import get_protocol, FileFinder
preprocessors = {"audio": FileFinder()}
cow_audio = get_protocol('My_datasets_specified.SpeakerDiarization.Detection', 
                   preprocessors=preprocessors)

'My_datasets_specified.SpeakerDiarization.Detection' found in /mnt/e/Files/Acoustic_Data/Datasets_specified/yaml/My_Databases_specified.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.


Load sound_event_detection task

In [2]:
from utils import sound_event_detection
# Defining Task Parameters
task = sound_event_detection.SoundEventDetection(
    protocol=cow_audio, 
    duration=1.0, 
    batch_size=32,
    classes= ["rumination","hoofbeat","breath","gulp","grazing"]
)

/home/wuxiaohui9707/miniconda3/envs/myenv/lib/python3.8/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/home/wuxiaohui9707/miniconda3/envs/myenv/lib/python3.8/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


Protocol My_datasets_specified.SpeakerDiarization.Detection does not precompute the output of torchaudio.info(): adding a 'torchaudio.info' preprocessor for you to speed up dataloaders. See pyannote.database documentation on how to do that yourself.


In [5]:
from pyannote.audio.models import custom_model
model = custom_model.MySoundEventDetection(task=task)
model.to("cuda")

MySoundEventDetection(
  (mfcc): MFCC(
    (amplitude_to_DB): AmplitudeToDB()
    (MelSpectrogram): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (linear1): Linear(in_features=40, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=16, bias=True)
)

In [4]:
import pytorch_lightning as pl
output_directory = os.path.join('/mnt/', 'e', 'Files', 'Acoustic_Data', 'Datasets_specified')
trainer = pl.Trainer(devices=1, accelerator="gpu", max_epochs=10, default_root_dir=output_directory)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params | In sizes      | Out sizes     
----------------------------------------------------------------------------------------
0 | mfcc              | MFCC             | 0      | [1, 1, 16000] | [1, 1, 40, 81]
1 | linear1           | Linear           | 1.3 K  | [1, 81, 40]   | [1, 81, 32]   
2 | linear2           | Linear           | 396    | [1, 81, 32]   | [1, 81, 12]   
3 | classifier      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'size'